# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from create_tables import *

In [2]:
main()
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

creating database...


In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*events.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

def get_song_data_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_song_data_files('data/song_data')

In [5]:
filepaths = song_files
print(filepaths)

['/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json', '/home/workspace/data/song_data/A/B/C/TRABCTK128F934B224.json', '/home/workspace/data/song_data/A/B/C/TRABCUQ128E0783E2B.json', '/home/workspace/data/song_data/A/B/C/TRABCIX128F4265903.json', '/home/workspace/data/song_data/A/B/C/TRABCYE128F934CE1D.json', '/home/workspace/data/song_data/A/B/C/TRABCEC128F426456E.json', '/home/workspace/data/song_data/A/B/C/TRABCXB128F4286BD3.json', '/home/workspace/data/song_data/A/B/C/TRABCAJ12903CDFCC2.json', '/home/workspace/data/song_data/A/B/C/TRABCPZ128F4275C32.json', '/home/workspace/data/song_data/A/B/C/TRABCKL128F423A778.json', '/home/workspace/data/song_data/A/B/C/TRABCEI128F424C983.json', '/home/workspace/data/song_data/A/B/C/TRABCFL128F149BB0D.json', '/home/workspace/data/song_data/A/B/C/.ipynb_checkpoints/TRABCAJ12903CDFCC2-checkpoint.json', '/home/workspace/data/song_data/A/B/C/.ipynb_checkpoints/TRABCEI128F424C983-checkpoint.json', '/home/workspace/data/song_data/A/B/C/.ipyn

In [6]:
df = pd.DataFrame()
for filepath in filepaths:
    temp = pd.read_json(filepath, lines=True)
    df = df.append(temp, ignore_index=True)

df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR8IEZO1187B99055E,NaN,NaN,,Marc Shaiman,SOINLJW12A8C13314C,City Slickers,149.86404,2008
1,1,AR558FS1187FB45658,NaN,NaN,,40 Grit,SOGDBUF12A8C140FAA,Intro,75.67628,2003
2,1,ARVBRGZ1187FB4675A,NaN,NaN,,Gwen Stefani,SORRZGD12A6310DBC3,Harajuku Girls,290.55955,2004
3,1,ARNF6401187FB57032,40.79086,-73.96644,"New York, NY [Manhattan]",Sophie B. Hawkins,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,305.16200,1994
4,1,AREVWGE1187B9B890A,-13.44200,-41.99520,Noci (BA),Bitter End,SOFCHDR12AB01866EF,Living Hell,282.43546,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()

song_data = song_data


In [8]:
import numpy as np
df.fillna("", inplace=True)
df.dropna(axis='index', how='any', subset=['artist_id', 'song_id', 'duration'], inplace=True)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
for row in song_data:
    print(row)
    cur.execute(song_table_insert, row)
    conn.commit()

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]


IntegrityError: insert or update on table "song" violates foreign key constraint "song_artist_id_fkey"
DETAIL:  Key (artist_id)=(AR8IEZO1187B99055E) is not present in table "artist".


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()
df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
for row in artist_data:
    print(row)
    cur.execute(artist_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files('data/log_data')

In [ ]:
filepath = log_files
filepath

In [ ]:
df = pd.DataFrame()
for file in filepath:
    df = df.append(pd.read_json(file, lines=True), ignore_index=True)

df.info() #8056
df.head()
df.fillna(np.nan, inplace=True)
df.dropna(axis='index', how='any', subset=['artist', 'length', 'song'], inplace=True)
df.info()
df[(df['registration'] == 1540919166796.0)]
df[(df['length'] < 1)]
df.dtypes

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
filt = (df['page'] == 'NextSong')
df_next_song = df[filt]
df_next_song.head()
df_next_song.shape

In [ ]:
t = pd.to_datetime(df_next_song['ts'], unit='ms')
t.head(5)

In [ ]:
time_data = pd.concat([t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday], axis=1)
time_data
column_labels = ("start_time", "hour", "day", "weekofyear", "month", "year", "weekday")

In [ ]:
time_df = pd.DataFrame(data=time_data.values, columns=column_labels)
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
df.columns
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df[user_df['userId'] == 24]
user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
#     print(f"{row.ts}|{row.userId}|{row.level}|{row.song}|{row.artist}|{row.sessionId}|{row.location}|{row.userAgent}|{row.length}|")
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
#     print(results)
    
    if results:
        print(row)
        print(results)
        songid, artistid = results
        
        # insert songplay record
        songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
    else:
        songid, artistid = None, None


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.